In [1]:
from IPython.display import Image

- fsdp
    - fully-sharded data parallel：全分片数据并行；
- data parallel
    - 每个计算设备都有整个神经网络模型的完整副本（Model Replica）
    - 假设一个批次的训练样本数为 $N$，使用 $M$ 个计算设备并行计算，每个计算设备会分配到 $N/M$ 个样本
        - 前向计算完成后，每个计算设备都会根据本地样本计算损失误差得到梯度 $G_i$（i 为加速卡编号）
        - 并将本地梯度 $G_i$ 进行广播（nccl，broadcast）
        - 所有计算设备需要聚合其他加速度卡给出的梯度值，然后使用平均梯度 $\frac{\sum G_i}{M}$ 对模型进行更新

### pipeline parallel

In [4]:
# https://arxiv.org/pdf/1811.06965
Image(url='../../imgs/gpipe-pp.png', width=700)

- 计算图中的下游设备（Downstream Device）需要长时间持续处于空闲状态，等待上游设备（Upstream Device）的计算完成，才能开始计算自身的任务。这种情况导致了设备的平均使用率大幅降低，形成了模型并行气泡（Model Parallelism Bubble），也称为流水线气泡（Pipeline Bubble）。
- (c) Pipeline parallelism divides the input mini-batch into smaller micro-batches, enabling different accelerators to
work on different micro-batches simultaneously.

### Optimizer states

In [7]:
Image(url='../../imgs/adam-mixed-precision.png', width=500)

混合精度优化的过程如上图。Adam优化器状态包括（都采用FP32保存式存储），假设模型参数量为$\Phi$，

- 模型参数备份：$4\Phi$
- 一阶动量：$4\Phi$
- 二阶动量：$4\Phi$
  
模型参数和梯度都是用FP16 格式存储（$2\Phi+2\Phi$），则共需要 2Φ + 2Φ + (4Φ + 4Φ + 4Φ) = 16Φ 字节存储

75 亿参数模型（7.5B），如果用FP16 格式，只需要 15GB 计算设备内存（7.5\*2），但是在训练阶段，模型状态实际上需要耗费120GB（7.5\*16）。

In [8]:
Image(url='https://www.microsoft.com/en-us/research/uploads/prodnew/2020/02/DeepSpeed-Image-1.png', width=500)

### 3D parallel

- https://huggingface.co/docs/transformers/v4.17.0/en/parallelism
- https://www.microsoft.com/en-us/research/blog/deepspeed-extreme-scale-model-training-for-everyone/

In [9]:
Image(url='https://www.microsoft.com/en-us/research/uploads/prodnew/2020/09/Blog_DeepSpeed3_Figure-1_highres-1536x923.png', width=500)

32个计算设备进行3D并行的例子。神经网络的各层分为4个流水线阶段。每个流水线阶段中的层在4个张量并行计算设备之间进一步划分。最后，每个流水线阶段有两个数据并行实例，使用ZeRO 内存优化在这2个副本之间划分优化器状态量。

In [10]:
Image(url='https://www.microsoft.com/en-us/research/uploads/prodnew/2020/09/Blog_DeepSpeed3_Figure2_highres.png', width=500)

### BLOOM

BLOOM 使用Megatron-DeepSpeed[108] 框架进行训练，主要包含两个部分：

- Megatron-LM提供张量并行能力和数据加载原语；
- DeepSpeed[138] 提供ZeRO 优化器、模型流水线及常规的分布式训练组件。
  
通过这种方式可以实现数据、张量和流水线三维并行。

In [6]:
Image(url='../../imgs/bloom.png', width=700)